# Exerice #3 in Financial DS - properties about etfs and selection of the best


In [2]:
#importing the libraries
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from openpyxl import Workbook
import requests
import re
from time import sleep
import random
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import yfinance as yf
import json
import csv

In [11]:
#setting the needed options
path = "C:\\Users\\eitan\\Downloads\\chromedriver_win32_new\\chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("excludeSwitches", ['enable-automation'])
driver = webdriver.Chrome(path, options=chrome_options)
chrome_options.add_argument("disable-infobars")
chrome_options.add_argument('--no-sandbox')
chrome = webdriver.Chrome(path, options=chrome_options)

In [12]:
#Collecting the properties for each etf of the list
def properties(etf):
    url = f'https://www.etf.com/{etf}#fit'
    path = "C:\\Users\\eitan\\Downloads\\chromedriver_win32\\chromedriver.exe"
    driver = webdriver.Chrome(path)
    driver.get(url)
    sleep(random.randint(40, 45))
    pe = driver.find_element_by_css_selector("#fundSharedHoldings+ .no-padding .rowText:nth-child(5) .pl0+ .pr0 .text-right").text
    sector_list = []
    rate = []
    sector = driver.find_elements_by_css_selector(".col-md-6 .column1 , .col-md-6 .pl0")
    for k in sector:
        g = k.text
        sector_list.append(g)  
    sector_list = sector_list[1:]
    sector_list, rate = sector_list[::2], sector_list[1::2]
    sector_df = pd.DataFrame(list(zip(sector_list, rate)),
                             columns =['sector', 'rate'])

    holdings = driver.find_element_by_css_selector('#fundSharedHoldings+ .no-padding .first+ .rowText .pl0+ .pr0 .text-right').text
    holdings = re.sub("[^0-9]", "",  holdings)
    holdings = int(holdings)
    holdings = float(holdings)
    geo_list = []
    geo = driver.find_elements_by_css_selector(".top10Holdings+ .pl0 .col-xs-12 .pl0+ .pr0 , .top10Holdings+ .pl0 .col-xs-12 .pl0")
    for i in geo:
        c = i.text
        geo_list.append(c)  

    geo_list, rate = geo_list[::2], geo_list[1::2]
    geo_df = pd.DataFrame(list(zip(geo_list, rate)),
                             columns =['geography', 'rate'])
    driver.close()
    return pe, holdings, sector_df, geo_df

    

In [3]:
#givimg the list of etfs
list_etfs = ["IJH","VBR","VBK","SCHA","IWO","IJJ","SPMD","XMLV","MDYV","SLY","VIOO","IVOO","SMMV","REGL","FNX","EZM","JKK","JHSC","IVOV","AVUV","SILJ","VTWG","PTMC","AVDV","JKL","RWK","SBIO","JKJ","MFMS","XSMO","OUSM","EWZS","SGDJ","XSVM","XMHQ"]
len(list_etfs)

35

In [13]:
#generating lists in order to keep the data I collect from the site
def making_lists(lst):
    sector_list = []
    pe_list= []
    geo_list= []
    holding_list = []
    sec = 'sector_df_'
    geo = 'geo_df_'
    p = 'pe_'
    h  = 'holdings_'
    for key in list_etfs:
        s = str(sec+key)
        sector_list.append(s)
        ho = str(h+key)
        holding_list.append(ho)
        g= str(geo+key)
        geo_list.append(g)
        pe = str(p+key)
        pe_list.append(ho)
    return sector_list, holding_list, geo_list, pe_list 

sector_list, holding_list, geo_list, pe_list = making_lists(list_etfs)
sector_list

['sector_df_IJH',
 'sector_df_VBR',
 'sector_df_VBK',
 'sector_df_SCHA',
 'sector_df_IWO',
 'sector_df_IJJ',
 'sector_df_SPMD',
 'sector_df_XMLV',
 'sector_df_MDYV',
 'sector_df_SLY',
 'sector_df_VIOO',
 'sector_df_IVOO',
 'sector_df_SMMV',
 'sector_df_REGL',
 'sector_df_FNX',
 'sector_df_EZM',
 'sector_df_JKK',
 'sector_df_JHSC',
 'sector_df_IVOV',
 'sector_df_AVUV',
 'sector_df_SILJ',
 'sector_df_VTWG',
 'sector_df_PTMC',
 'sector_df_AVDV',
 'sector_df_JKL',
 'sector_df_RWK',
 'sector_df_SBIO',
 'sector_df_JKJ',
 'sector_df_MFMS',
 'sector_df_XSMO',
 'sector_df_OUSM',
 'sector_df_EWZS',
 'sector_df_SGDJ',
 'sector_df_XSVM',
 'sector_df_XMHQ']

In [14]:
#collecting the data for all the etfs and saving the pe and holdings so I don't need to run again
def calling(list_etfs, sector_list, holding_list, geo_list, pe_list): 
    geo_dfs = {}
    sector_dfs = {}
    for i in range(0,len(list_etfs)):
        pe_list[i], holding_list[i], sector_list[i], geo_list[i] = properties(list_etfs[i])
        key_ = str(list_etfs[i])
        geo_dfs[i] = {key_ : pd.DataFrame(geo_list[i])}
        sector_dfs[i] = {key_ : pd.DataFrame(sector_list[i])}
    pe_holdings_df = pd.DataFrame(list(zip(list_etfs, pe_list, holding_list)),
                             columns =['etf', 'average pe', 'holdings'])
    return pe_holdings_df, geo_dfs, sector_dfs

pe_holdings_df, geo_dfs, sector_dfs = calling(list_etfs, sector_list, holding_list, geo_list, pe_list)
pe_holdings_df.to_csv('pe_holdings_df')

#saving the data collected about the sectors and geography spread in a json file so I don't need to run again
class JSONEncoder(json.JSONEncoder):
    def default(self, sector_dfs):
        if hasattr(sector_dfs, 'to_json'):
            return sector_dfs.to_json(orient='records')
        return json.JSONEncoder.default(self, sector_dfs)
with open('sector.json', 'w') as fp:
    json.dump(sector_dfs, fp, cls=JSONEncoder)
with open('geography.json', 'w') as fp:
    json.dump(geo_dfs, fp, cls=JSONEncoder)
    
pe_holdings_df

,etf,average pe,holdings
0,IJH,56.15,402.0
1,VBR,-54.00,907.0
2,VBK,"4,183.11",589.0
3,SCHA,-37.98,1747.0
4,IWO,-63.70,1092.0
5,IJJ,82.59,295.0
6,SPMD,56.08,401.0
7,XMLV,28.35,82.0
8,MDYV,83.85,296.0
9,SLY,-29.97,602.0


In [68]:
geo_dfs

{0: {'IJH':        geography     rate
  0  United States  100.00%}, 1: {'VBR':        geography    rate
  0  United States  99.93%
  1         Canada   0.07%}, 2: {'VBK':        geography     rate
  0  United States  100.00%
  1         Canada       --}, 3: {'SCHA':        geography    rate
  0  United States  99.94%
  1         Canada   0.06%}, 4: {'IWO':        geography    rate
  0  United States  99.63%
  1         Canada   0.37%}, 5: {'IJJ':        geography     rate
  0  United States  100.00%}, 6: {'SPMD':        geography     rate
  0  United States  100.00%}, 7: {'XMLV':        geography     rate
  0  United States  100.00%}, 8: {'MDYV':        geography     rate
  0  United States  100.00%}, 9: {'SLY':        geography     rate
  0  United States  100.00%
  1         Canada       --}, 10: {'VIOO':        geography     rate
  0  United States  100.00%
  1         Canada       --}, 11: {'IVOO':        geography     rate
  0  United States  100.00%}, 12: {'SMMV':        geograph

In [69]:
sector_dfs

{0: {'IJH':                         sector    rate
  0                   Financials  23.04%
  1                  Industrials  17.84%
  2           Consumer Cyclicals  16.65%
  3                   Technology  12.49%
  4                   Healthcare  11.27%
  5              Basic Materials   6.19%
  6       Consumer Non-Cyclicals   4.58%
  7                    Utilities   3.85%
  8                       Energy   3.83%
  9  Telecommunications Services   0.28%},
 1: {'VBR':                         sector    rate
  0                   Financials  30.03%
  1                  Industrials  16.51%
  2           Consumer Cyclicals  15.20%
  3                   Technology   9.33%
  4                   Healthcare   8.15%
  5              Basic Materials   7.71%
  6                    Utilities   5.40%
  7       Consumer Non-Cyclicals   4.24%
  8                       Energy   2.98%
  9  Telecommunications Services   0.36%},
 2: {'VBK':                         sector    rate
  0                   T

In [39]:
#calling the data again
f = open('sector.json',) 
data_sector = json.load(f)

g = open('geography.json',) 
data_geography = json.load(g)
data_geography

{'0': {'IJH': '[{"geography":"United States","rate":"100.00%"}]'},
 '1': {'VBR': '[{"geography":"United States","rate":"99.93%"},{"geography":"Canada","rate":"0.07%"}]'},
 '2': {'VBK': '[{"geography":"United States","rate":"100.00%"},{"geography":"Canada","rate":"--"}]'},
 '3': {'SCHA': '[{"geography":"United States","rate":"99.94%"},{"geography":"Canada","rate":"0.06%"}]'},
 '4': {'IWO': '[{"geography":"United States","rate":"99.63%"},{"geography":"Canada","rate":"0.37%"}]'},
 '5': {'IJJ': '[{"geography":"United States","rate":"100.00%"}]'},
 '6': {'SPMD': '[{"geography":"United States","rate":"100.00%"}]'},
 '7': {'XMLV': '[{"geography":"United States","rate":"100.00%"}]'},
 '8': {'MDYV': '[{"geography":"United States","rate":"100.00%"}]'},
 '9': {'SLY': '[{"geography":"United States","rate":"100.00%"},{"geography":"Canada","rate":"--"}]'},
 '10': {'VIOO': '[{"geography":"United States","rate":"100.00%"},{"geography":"Canada","rate":"--"}]'},
 '11': {'IVOO': '[{"geography":"United St

In [6]:
#making a data frame for the results
    
#collecting the change data and finding the interest dates and periods with keeping the results in a data frame
m1,m3,m6,y1,y2,y5=[],[],[],[],[],[]
for i in range(0,len(list_etfs)):
    sleep(random.randint(3, 5))
    tick = str(list_etfs[i])
    #printing where I am
    print(tick) 

    y = yf.Ticker(tick)
    ydf = y.history(start="2007-01-01", end="2020-11-30", interval="1mo")
    ydf['Date'] = ydf.index
    
    m1.append(ydf['Close'].pct_change().shift(1).iloc[-1])
    m3.append(ydf['Close'].pct_change().shift(3).iloc[-1])
    m6.append(ydf['Close'].pct_change().shift(6).iloc[-1])
    y1.append(ydf['Close'].pct_change().shift(12).iloc[-1])
    y2.append(ydf['Close'].pct_change().shift(24).iloc[-1])
    y5.append(ydf['Close'].pct_change().shift(60).iloc[-1])
    
rate_growth_df = pd.DataFrame(list(zip(list_etfs, m1, m3, m6, y1, y2, y5)),
                             columns =['etf', "1 month change",  "3 month change", "6 month change", "1 year change", "2 year change", "5 year change"])
rate_growth_df.to_csv('rate_growth_df')
rate_growth_df

IJH
VBR
VBK
SCHA
IWO
IJJ
SPMD
XMLV
MDYV
SLY
VIOO
IVOO
SMMV
REGL
FNX
EZM
JKK
JHSC
IVOV
AVUV
SILJ
VTWG
PTMC
AVDV
JKL
RWK
SBIO
JKJ
MFMS
XSMO
OUSM
EWZS
SGDJ
XSVM
XMHQ


,etf,1 month change,3 month change,6 month change,1 year change,2 year change,5 year change
0,IJH,0.027114,-0.037701,0.000000,-0.094832,-0.079994,0.025450
1,VBR,0.037335,-0.042579,0.000000,-0.101774,-0.081340,0.020117
2,VBK,0.010064,-0.016474,0.000000,-0.067552,-0.063685,0.026770
3,SCHA,0.026208,-0.040315,0.000000,-0.083958,-0.074100,-0.005369
4,IWO,0.008954,-0.022073,0.000000,-0.070569,-0.073591,0.025666
5,IJJ,0.000000,0.000000,0.036629,-0.250955,0.074161,-0.010201
6,SPMD,0.024601,-0.035471,0.000000,-0.093954,-0.081456,-0.007734
7,XMLV,-0.001200,0.019635,-0.025665,0.005549,0.034131,0.000000
8,MDYV,0.036766,-0.045535,0.000000,-0.104892,-0.096475,-0.010375
9,SLY,0.025507,-0.047716,0.000000,-0.095101,-0.086404,-0.006877


In [71]:
rate_growth_df.nlargest(n=1, columns="1 month change")

,etf,1 month change,3 month change,6 month change,1 year change,2 year change,5 year change
26,SBIO,0.061464,0.036546,0.088382,0.069027,0.0,-0.079755


In [72]:
rate_growth_df.nlargest(n=1, columns="3 month change")

,etf,1 month change,3 month change,6 month change,1 year change,2 year change,5 year change
28,MFMS,0.048022,0.086881,0.124898,-0.013565,-0.106413,NaN


In [73]:
rate_growth_df.nlargest(n=1, columns="6 month change")

,etf,1 month change,3 month change,6 month change,1 year change,2 year change,5 year change
20,SILJ,-0.007965,0.02694,0.262032,0.181386,0.147352,0.283186


In [74]:
rate_growth_df.nlargest(n=1, columns="1 year change")

,etf,1 month change,3 month change,6 month change,1 year change,2 year change,5 year change
20,SILJ,-0.007965,0.02694,0.262032,0.181386,0.147352,0.283186


In [75]:
rate_growth_df.nlargest(n=1, columns="2 year change")

,etf,1 month change,3 month change,6 month change,1 year change,2 year change,5 year change
32,SGDJ,0.000449,0.03914,0.139803,0.120156,0.157052,0.372489


In [76]:
rate_growth_df.nlargest(n=1, columns="5 year change")

,etf,1 month change,3 month change,6 month change,1 year change,2 year change,5 year change
32,SGDJ,0.000449,0.03914,0.139803,0.120156,0.157052,0.372489


In [4]:
rate_growth_df = pd.read_csv("rate_growth_df.csv")
rate_growth_df.nlargest(n=10, columns="6 month change change")

FileNotFoundError: [Errno 2] File b'rate_growth_df.csv' does not exist: b'rate_growth_df.csv'

In [23]:
rate_growth_df.nsmallest(n=10, columns="1 year change")

,etf,1 month change,3 month change,6 month change,1 year change,2 year change,5 year change
5,IJJ,0.000000,0.000000,0.036629,-0.250955,0.074161,-0.010201
19,AVUV,0.050770,-0.049712,0.000000,-0.112722,NaN,NaN
24,JKL,0.039424,-0.048776,0.000000,-0.108228,-0.096568,0.015006
15,EZM,0.037187,-0.036087,0.000000,-0.105601,-0.094344,0.000000
8,MDYV,0.036766,-0.045535,0.000000,-0.104892,-0.096475,-0.010375
13,REGL,0.034455,-0.045675,0.000000,-0.104078,-0.056783,0.016419
1,VBR,0.037335,-0.042579,0.000000,-0.101774,-0.081340,0.020117
27,JKJ,0.020058,-0.038268,0.000000,-0.101289,-0.075295,0.019497
9,SLY,0.025507,-0.047716,0.000000,-0.095101,-0.086404,-0.006877
0,IJH,0.027114,-0.037701,0.000000,-0.094832,-0.079994,0.025450


Outlook for 2021:

I would suggest to invest in these three etfs:

1. IVOV
2. VIOO

These two because they did very well in the past five years (#3,4 out of the list), with the past year not being that good, therefore I expect a recovery in the coming year. Furthermore they have a significant percentage of holdings in the financial sector that is expected to go up. 

3. VBK

Did well (#9 out of the list) in the past five years and has a high percent of holdings in healthcare with some financials and a bit in the energy sector. It was a tough choice between JKK and VBK, I preferred VBK since the past year went worse,  therefore I expect they will have a more steap growth. JKK had a better year with decreasing gains, while VBK had a worse year but we can see recovery.